## step 1: If we need to consume a local build of [dotnet/interactive](https://github.com/dotnet/interactive) repo, run the following cell

In [ ]:
$dotnetInteractivePath="D:\microsoft\dotnet\interactive"
if ($IsWindows) {
    Invoke-Expression "$dotnetInteractivePath\build-js.cmd"
}else {
    Invoke-Expression "$dotnetInteractivePath\build-js.sh"
}

## step 2: Create the NuGet package CSharpProject
The following script will copy the generated packages into from C:\temp\packages\

In [ ]:
Invoke-Expression "$dotnetInteractivePath\repack.ps1"

## step 3: Create a new NuGet Feed in try dotnet pointing to this path

## step 4: Build of the JavaScript code in [trydotnet service](https://github.com/dotnet/try) repo

In [1]:
$tryDotnetPath="D:\microsoft\dotnet\try"
if ($IsWindows) {
    Invoke-Expression "$tryDotnetPath\build-js.cmd"
}else {
    Invoke-Expression "$tryDotnetPath\build-js.sh"
}

Building NPM in directory src\microsoft-trydotnet


Error: Command cancelled.

## step 5: Build of trydotnet service

In [ ]:
Invoke-Expression "$tryDotnetPath\build.cmd"

## step 6: Open try dotnet in Visual Studio and set Microsoft.TryDotNet as a startup project

## step 7: Run try dotnet (F5)

## step 8: code src\microsoft-learn-mock and then open menu Terminal and start a new terminal

## step 9:

npm ci

npm run buildProd

npx http-server

http://192.168.1.68:8080/site/DocsHost.html?trydotnetUrl=https://localhost:7061/api/trydotnet.min.js&trydotnetOrigin=https://localhost:7061/

converted to url:

http://192.168.1.68:8080/site/DocsHost.html?trydotnetUrl=https%3A%2F%2Flocalhost%3A7061%2Fapi%2Ftrydotnet.min.js&trydotnetOrigin=https%3A%2F%2Flocalhost%3A7061%2F

## Try .NET / Microsoft Learn flow diagram

The diagram available through the link below provides a comprehensive of the interaction between Try .NET service (trydotnet.microsoft.com) and a host page (learn.microsoft.com).

https://github.com/dotnet/interactive/tree/main/src/Microsoft.DotNet.Interactive.CSharpProject#try-net--microsoft-learn-flow

The core of Try .NET is powered by the `CSharpProjectKernel`, which serves as the backend for .NET Interactive.